In [111]:
%load_ext autoreload
%autoreload 2

import os, sys
import copy
import socket
from tqdm import tqdm
import torch
import pickle
from torch import optim
from torch.utils.tensorboard import SummaryWriter

#!pip install networkx matplotlib
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from libs.topology_manager import *
from libs import fl, nn, data, log

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
#log.init("info", "federated.log")
#log.init("debug", "flkafka.log")

In [113]:
adj_mat = np.array([[0,1,1,1,1,1],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0], 
             [1,0,0,0,0,0]])

node_types = {'aggregator': [0,1], 'trainer': [2,3], 'broadcaster': [4,5]}

di_graph = nx.DiGraph(adj_mat)
#nx.draw(di_graph)

In [114]:
class FedArgs():
    def __init__(self):
        self.num_clients = len(adj_mat)
        self.epochs = 10
        self.local_rounds = 1
        self.client_batch_size = 32
        self.test_batch_size = 128
        self.learning_rate = 1e-4
        self.weight_decay = 1e-5
        self.cuda = False
        self.seed = 1
        self.topic = 'pyflx'
        self.tb = SummaryWriter('../out/runs/flkafka', comment="Mnist Distributed Federated training")

fedargs = FedArgs()

RuntimeError: can't start new thread

In [115]:
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [106]:
host = socket.gethostname()
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]
ctp = DistributedTopology(adj_mat, clients, node_types)

In [107]:
#Initialize Global and Client models
global_model = nn.ModelMNIST()
client_models = {client: copy.deepcopy(global_model) for client in clients}

# Function for training
def train_model(model, train_loader, fedargs, device):
    model, loss = fl.client_update(model,
                                train_loader,
                                fedargs.learning_rate,
                                fedargs.weight_decay,
                                fedargs.local_rounds,
                                device)
    return model, loss

In [108]:
# Load MNIST Data to clients
train_data, test_data = data.load_dataset("mnist")
clients_data = data.split_data(train_data, clients)
client_train_loaders, client_test_loaders = data.load_client_data(clients_data, fedargs.client_batch_size, 0.2, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

clients_info = {
        client: {"train_loader": client_train_loaders[client],
                 "test_loader": client_test_loaders[client]}
        for client in clients
    }

In [109]:
import asyncio
import time

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def process(client, epoch, ctp, model, train_loader, test_loader, fedargs, device):
    log.info("Processing Client {}".format(client))
    
    # Consume and Average, epoch passed is actually prev epoch, for which we want to consume updates
    rcvd_models = ctp.consume_model(client, fedargs.topic, model, epoch - 1)
    log.info("Client {} received {} model updates from {}".format(client, len(rcvd_models), list(rcvd_models.keys())))
    if len(rcvd_models) != 0:
        model = fl.federated_avg(rcvd_models)

    # Train    
    model, loss = train_model(model, train_loader, fedargs, device)
    
    # Publish
    ctp.produce_model(client, fedargs.topic, model, epoch)

    # Plot and Log
    for local_epoch, loss in enumerate(list(loss.values())):
        fedargs.tb.add_scalars("Training Loss/" + client, {str(epoch): loss}, str(local_epoch + 1))

    log.jsondebug(loss, "Epoch {} of {} : Federated Training loss, Client {}".format(epoch, fedargs.epochs, client))
    log.modeldebug(model, "Epoch {} of {} : Client {} Update".format(epoch, fedargs.epochs, client))

    # Test
    test_output = fl.eval(model, test_loader, device)
    fedargs.tb.add_scalar("Accuracy/" + client, test_output["accuracy"], epoch)
    log.jsoninfo(test_output, "Test Outut after Epoch {} of {} for Client {}".format(epoch, fedargs.epochs, client))

In [110]:
import time
start_time = time.time()

# Federated Training
for _epoch in tqdm(range(fedargs.epochs)):

    epoch = _epoch + 1
    log.info("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))

    # Clients  
    tasks = [process(client, epoch, ctp, client_models[client],
                     clients_info[client]['train_loader'],
                     clients_info[client]['test_loader'],
                     fedargs, device) for client in clients]
    await asyncio.wait(tasks)
    
print(time.time() - start_time)

  0%|          | 0/10 [00:00<?, ?it/s]2021-08-16 23:19:20,499 - <ipython-input-110-c8ea6acd90e4>::<module>(l:8) : Federated Training Epoch 1 of 10 [MainProcess : MainThread (INFO)]
2021-08-16 23:19:20,538 - <ipython-input-109-7a5cd7c1f6e3>::process(l:12) : Processing Client bladecluster.iitp.org(3) [MainProcess : asyncio_7 (INFO)]
2021-08-16 23:19:20,548 - <ipython-input-109-7a5cd7c1f6e3>::process(l:12) : Processing Client bladecluster.iitp.org(2) [MainProcess : asyncio_11 (INFO)]
2021-08-16 23:19:20,553 - <ipython-input-109-7a5cd7c1f6e3>::process(l:12) : Processing Client bladecluster.iitp.org(4) [MainProcess : asyncio_8 (INFO)]
2021-08-16 23:19:20,558 - <ipython-input-109-7a5cd7c1f6e3>::process(l:12) : Processing Client bladecluster.iitp.org(1) [MainProcess : asyncio_6 (INFO)]
2021-08-16 23:19:20,564 - <ipython-input-109-7a5cd7c1f6e3>::process(l:12) : Processing Client bladecluster.iitp.org(6) [MainProcess : asyncio_9 (INFO)]
2021-08-16 23:19:20,578 - <ipython-input-109-7a5cd7c1f6e3>

2021-08-16 23:19:46,029 - <ipython-input-109-7a5cd7c1f6e3>::process(l:36) : Test Outut after Epoch 1 of 10 for Client bladecluster.iitp.org(2) {
    "accuracy": 90.3,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1806,
    "test_loss": 0.3241649172306061
} [MainProcess : asyncio_11 (INFO)]
2021-08-16 23:19:46,045 - <ipython-input-109-7a5cd7c1f6e3>::process(l:36) : Test Outut after Epoch 1 of 10 for Client bladecluster.iitp.org(6) {
    "accuracy": 90.35,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1807,
    "test_loss": 0.3523443686962128
} [MainProcess : asyncio_9 (INFO)]
2021-08-16 23:19:46,096 - <ipython-input-109-7a5cd7c1f6e3>::process(l:36) : Test Outut after Epoch 1 of 10 for

2021-08-16 23:20:20,728 - <ipython-input-109-7a5cd7c1f6e3>::process(l:36) : Test Outut after Epoch 2 of 10 for Client bladecluster.iitp.org(3) {
    "accuracy": 91.05,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1821,
    "test_loss": 0.32294040703773497
} [MainProcess : asyncio_2 (INFO)]
2021-08-16 23:20:20,943 - <ipython-input-109-7a5cd7c1f6e3>::process(l:36) : Test Outut after Epoch 2 of 10 for Client bladecluster.iitp.org(1) {
    "accuracy": 91.55,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1831,
    "test_loss": 0.3304774861335754
} [MainProcess : asyncio_3 (INFO)]
2021-08-16 23:20:21,175 - <ipython-input-109-7a5cd7c1f6e3>::process(l:36) : Test Outut after Epoch 2 of 10 fo

2021-08-16 23:20:55,060 - <ipython-input-109-7a5cd7c1f6e3>::process(l:36) : Test Outut after Epoch 3 of 10 for Client bladecluster.iitp.org(4) {
    "accuracy": 89.55,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1791,
    "test_loss": 0.3590500154495239
} [MainProcess : asyncio_10 (INFO)]
2021-08-16 23:20:55,141 - <ipython-input-109-7a5cd7c1f6e3>::process(l:36) : Test Outut after Epoch 3 of 10 for Client bladecluster.iitp.org(2) {
    "accuracy": 91.14999999999999,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1823,
    "test_loss": 0.3277583651542664
} [MainProcess : asyncio_9 (INFO)]
2021-08-16 23:20:55,228 - <ipython-input-109-7a5cd7c1f6e3>::process(l:36) : Test Outut after Epoc

2021-08-16 23:21:26,791 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(4)' successfully produced to pyflx [0] at offset 10 [MainProcess : asyncio_5 (INFO)]
2021-08-16 23:21:26,863 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_1 (INFO)]
2021-08-16 23:21:27,691 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(3)' successfully produced to pyflx [0] at offset 11 [MainProcess : asyncio_1 (INFO)]
2021-08-16 23:21:28,752 - <ipython-input-109-7a5cd7c1f6e3>::process(l:36) : Test Outut after Epoch 4 of 10 for Client bladecluster.iitp.org(6) {
    "accuracy": 92.95,
    "attack": {
        "attack_success_count": 0,
        "attack_success_rate": 0,
        "instances": 0,
        "misclassification_rate": 0,
        "misclassifications": 0
    },
    "correct": 1859,
    "test

2021-08-16 23:21:59,448 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_9 (INFO)]
2021-08-16 23:21:59,498 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(3)' successfully produced to pyflx [0] at offset 14 [MainProcess : asyncio_6 (INFO)]
2021-08-16 23:21:59,600 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : asyncio_10 (INFO)]
2021-08-16 23:22:00,020 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(2)' successfully produced to pyflx [0] at offset 15 [MainProcess : asyncio_9 (INFO)]
2021-08-16 23:22:00,022 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx. ^C to exit. [MainProcess : asyncio_8 (INFO)]
2021-08-16 23:22:00,210 - /home/harsh_192

CancelledError: 